#### Writers Note
Below is a demonstration on what `cadr` and `caddr` do.

In [13]:
(define (cadr s) (car (cdr s)))
(define (caddr s) (cadr (cdr s)))

In [14]:
(define x (list 1 2 3 4))
(cadr x)

2

In [15]:
(caddr x)

3

# Differentiation

The following problems develop a system for [symbolic differentiation](https://en.wikipedia.org/wiki/Differentiation_rules#Elementary_rules_of_differentiation) of algebraic expressions. The `derive` Scheme procedure takes an algebraic expression and a variable and returns the derivative of the expression with respect to the variable. Symbolic differentiation is of special historical significance in Lisp. It was one of the motivating examples behind the development of the language. **Differentiationg is a recursive process that applies different rules to different kinds of expressions**.

In [2]:
; Returns the derivative of 'expr' with respect to 'var'
(define (derive expr var)
  (cond
   ((number? expr) 0 ) ; If expr is a number, then the derivative is just 0
   ; If 'expr' is a variable and if it's the same as 'var' then return 1, 
   ; otherwise return 0.
   ; For example, if `expr` is x, and we derive it with respect to x, then
   ; the derivative is 1. Otherwise, if 'expr' is y, then the derivative is 0
   ((variable? expr) (if (same-variable? expr var) 1 0))
   ((sum? expr) (derive-sum expr var))
   ((product? expr) (derive-product expr var))
   ((exp? expr) (derive-exp expr var))
   (else 'Error)))

To implement the system, we will use the following data abstraction. Sums and products are lists, and they are simplified on construction.

In [3]:
; Variables are represented as symbols
(define (variable? x) (symbol? x))

; Checks if v1 is the same as v2
(define (same-variable? v1 v2)
  (and (variable? v1) (variable? v2) (eq? v1 v2)))

In [4]:
; Numbers are compared with =
(define (=number? expr num)
  ; Checks if 'expr' is a numbe
  (and (number? expr) (= expr num)))

In [5]:
; Sums are represented as lists that start with +
(define (make-sum a1 a2)
  (cond
   ((=number? a1 0) a2)
   ((=number? a2 0) a1)
   ((and (number? a1) (number? a2)) (+ a1 a2))
   (else (list '+ a1 a2))
   ) ; End of cond
  ) ; End of define

; Checks if x is a symbol and the first element is a '+'
(define (sum? x)
  (and (list? x) (eq? (car x) '+)))

(define (addend s) (cadr s))
(define (augend s) (caddr s))

In [6]:
; Products are represented as lists that start with *
(define (make-product m1 m2)
  (cond
   ; If either m1 or m2 is 0, then the product i s0
   ((or (=number? m1 0) (=number? m2 0)) 0)
   ((=number? m1 1) m2) ; If one of them
   ((=number? m2 1) m1)
   ((and (number? m1) (number? m2)) (* m1 m2))
   (else (list '* m1 m2))))

; Checks if x is a list and if the first element is a '*'
(define (product? x)
  (and (list? x) (eq? (car x) '*)))

(define (multiplier p) (cadr p))
(define (multiplicand p) (caddr p))

## Q3: Derive Sum
Implement `derive-sum`, a procedure that differentiates a sum by summing the derivatives of the `addend` and `augend`. Use data abstraction for a sum.

#### WWSD

In [7]:
(make-sum 1 3)
; Ans: 4

4

In [8]:
(make-sum 'x 0)
; Ans: x

x

In [9]:
(make-sum 0 'x)
; Ans: x

x

In [10]:
(make-sum 'a 'x)
; Ans: (+ a x)

(+ a x)

In [11]:
(make-sum 'a (make-sum 'x 1))
; Ans: (+ a (+ x 1))

(+ a (+ x 1))

In [12]:
(derive '(+ x 3) 'x)
; Ans: 1


Traceback (most recent call last):
  File "In [12]", line 1, col 1, in 'derive'
  File "In [2]", line 10, col 18
RunTimeError: unbound variable 'derive-sum'



#### Strategy

As the description says, we just sum the derivatives of `addend` and `augend`. We can evaluate the derivatives by calling the `derive` procedure,

In [ ]:
(derive (addend expr) var)
(derive (augend expr) var)

Then we can sum the derivatives above with the `make-sum` procedure. The implementation is as the following,

In [ ]:
(define (derive-sum expr var)
  (make-sum
    (derive (addend expr) var)
    (derive (augend expr) var)
  )
)

## Q4: Derive Product

Implement `derive-product`, which applies the **product rule** to differentiate products. This means taking the multiplier and multiplicand, and then summing the result of multiplying one by the derivative of the other.

#### WWSD

In [ ]:
(make-product 2 3)
; Ans: 0

In [ ]:
(make-product 'x 0)
; Ans: 0

In [ ]:
(make-product 1 'x)
; Ans: x

In [ ]:
(make-product 'a 'x)
; Ans: (* a x)

#### Strategy

As described by the problem description, first we'll have to take the derivative of the multiplier and multiplicand,

In [ ]:
(define dmul (derive (multiplier expr) var))
(define dcand (derive (multiplicand expr) var))

Then we'll have 2 different terms:

1. The product of multiplier derivative with the multiplicand
2. The product of multipler with multiplicand derivative

In [ ]:
(make-product dmul multiplicand)
(make-product multiplier dcand)

Then we sum the 2 results above,

In [ ]:
(make-sum
 (make-product dmul multiplicand)
 (make-product multiplier dcand)
 )

The implementation is as the following,

In [ ]:
(define (derive-product expr var)
  (begin
    (define dmul (derive (multiplier expr) var))
    (define dcand (derive (multiplicand expr) var)))
  (make-sum
    (make-product dmul (multiplicand expr))
    (make-product (multiplier expr) dcand)
  ); End of make-sum
) ; End of define

## Q5: Make Exp

Implement a data abstraction for exponentiation: a `base` raised to the power of an `exponent`. The `base` can be any expression, but assume that the `exponent` is a non-negative integer. You can simplify the case when `exponent` is `0` or `1`, or when `base` is a number, by returning numbers from constructor `make-exp`. In other cases, you can represent the exp as a triple `(^ base exponent)`.

#### WWSD

In [ ]:
scm> (define x^2 (make-exp 'x 2))
scm> (define x^3 (make-exp 'x 3))
scm> (make-exp 2 4)
; Ans: 16

In [ ]:
scm> (make-exp 'x 1)
; Ans: x

In [ ]:
scm> (make-exp 'x 0)
; Ans: 1

In [ ]:
scm> x^2
; Ans: (^ x 2)

In [ ]:
scm> (base x^2)
; Ans: 2

In [ ]:
scm> (exponent x^2)
; Ans: 2

In [ ]:
(exp? x^2) ; True or False
; Ans: True

In [ ]:
scm> (exp? 1)
; False

In [ ]:
scm> (exp? 'x)
; False

#### Strategy - `make-exp`

We know that `x^2` can be represented as `(^ x 2)` from the problem description and WWSD. This way, we can tell that the `exp` argument can be represented as `(^ base exponent)`. The ADT for `base` and `exponent` then can be defined as the second element and the third element of the list `(^ base exponent)`, respectively.

To define the `make-exp` procedure, we have the following 3 simple base cases:

1. If the `exponent` is `0`, then we just return 1
2. If the `exponent` is `1`, then we return the base
3. If the `base` is a number, then we return the base raised to the power of the `exponent`

In [ ]:
(cond
 ((= exponent 0) 1)
 ((= exponent 1) base)
 ((number? base) (expt base exponent))

Otherwise, the expression has to be a symbol and a exponential, which can be represented by `(^ base exponent)`.

In [ ]:
(else (list '^ base exponent))

The implementation looks like the following,

In [17]:
(define (make-exp base exponent)
  (cond
   ((= exponent 0) 1) ; If the exponent is 0, then just return 1
   ((= exponent 1) base) ; If the exponent is 1, then return the base
   ((number? base) (expt base exponent)) ; If the base is a number, then evaluate the base raised by the power of the exponent
   (else (list '^ base exponent)) ; Otherwise, create a list representation of (^ base exponent)
   ) ; End of cond
  ) ; End of define


Traceback (most recent call last):
  File "In [17]", line 6, col 97
ReadError: unexpected end of input



#### Strategy - `base` and `exponent`

Based on the definition of an exponential expression above, the `base` and the `exponent` are the 2nd and 3rd element of the list, respectively.

In [ ]:
(^ base exponent)

Thus, the ADT for both of them are simply the `cadr` and `caddr` of the list.

In [ ]:
(define (base exp) (cadr exp))
(define (exponent exp) (caddr exp))

#### Strategy - 'exp?`

To check whether an expression is an exponential expression, we simply check for 2 things:

1. Whether the `exp` is a list
2. Whether the first element, `car`, of the `exp` is a `^` symbol.

In [1]:
(define (exp? exp) (and
                    (list? exp)
                    (eq? (car exp) '^)
                    ) ; End of and
  ); End of define

## Q6: Derive Exp

Implement `derive-exp`, which uses the [power rule](https://en.wikipedia.org/wiki/Power_rule) to derive exponents. Reduce the power of the exponent by one, and multiply the entire expression by the original exponent.

#### WWSD

In [ ]:
scm> (define x^2 (make-exp 'x 2))
scm> (define x^3 (make-exp 'x 3))
scm> (derive x^2 'x) ; Remember products have the form (* a b)
; Ans: (* 2 x)

In [1]:
scm> (derive x^3 'x)
; Ans: (* 3 (^ x 2))


Traceback (most recent call last):
  File "In [1]", line 1, col 1
RunTimeError: unbound variable 'scm>'



#### Strategy
T

For the derivation, the base case is that if the `exponent` is 2, then the outcome is simply a product rule. Remember to use the data abstraction, the `make-product` procedure.

In [ ]:
(if (= 2 (exponent exp))
    (make-product (exponent exp) (base exp))

Otherwise, we have to combine product rule with the exponentiation form.

In [ ]:
(make-product (exponent exp) (make-exp (base exp) (- (exponent exp) 1)))